# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [6]:
# your code here
# your code here
import pandas as pd
import numpy as np

df = pd.read_csv('tic-tac-toe.csv')


df.head()
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [7]:
# your code here
from sklearn.preprocessing import LabelEncoder

# Perform one-hot encoding for each board position
# Generate dummy variables for each position
positions = ['TL','TM','TR','ML','MM','MR','BL','BM','BR']
for pos in positions:
    dummies = pd.get_dummies(df[pos], prefix=pos)
    df = pd.concat([df, dummies], axis=1)
    df.drop(pos, axis=1, inplace=True)

# Now the dataframe includes columns like TL_x, TL_o, TL_b, etc. for each position
# Encode the 'class' column (True/False to 1/0)
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class']) 

# Divide the data into features (X) and target (y)
X = df.drop('class', axis=1)
y = df['class']

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [8]:
# your code here
import tensorflow as tf


train_dataframe = X_train.copy()
train_dataframe['class'] = y_train
test_dataframe = X_test.copy() 
test_dataframe['class'] = y_test

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, 
                   epochs=50,
                   batch_size=32,
                   validation_split=0.2,
                   verbose=0)

# Evaluate the model's performance
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")

# Generate predictions on the test data
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

# Compare predictions with actual values
print("\nSample predictions vs actual values:")
comparison_dataframe = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predictions.flatten()
})
print(comparison_dataframe.head(10))



# Calculate and display accuracy metrics
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test, predictions))

# Save the model
print(f"\nTraining History:")
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}") 
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

c:\Users\IT Department\.conda\envs\iron_hack\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test accuracy: 0.9740
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

Sample predictions vs actual values:
     Actual  Predicted
836       0          0
477       1          1
350       1          1
891       0          0
855       0          0
261       1          1
215       1          1
334       1          0
86        1          1
876       0          0

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        67
           1       0.97      0.99      0.98       125

    accuracy                           0.97       192
   macro avg       0.98      0.97      0.97       192
weighted avg       0.97      0.97      0.97       192


Training History:
Final Training Loss: 0.0088
Final Training Accuracy: 1.0000
Final Validation Loss: 0.0781
Final Validation Accuracy: 0.9740


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [9]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [10]:
# your answer here